Assurez-vous avant toute chose d'obtenir une clé pour utiliser l'API de Airlabs à l'adresse ci-après : https://airlabs.co/docs/flights

Veuillez configurer un environnement virtuel et y installer l'environnement de développement indiqué dans le fichier requirements.txt

_______________________________________________________________________________________

Les données utilisées sont stockées dans des fichiers au format JSON répartis dans trois collections. 
Les tables suivantes décrivent le schéma défini pour chaque collection de jeux de données : AIRLINES ; DEPARTURES ; FLIGHTS.

Veuillez vous référer au document des spécifications de conception logiciel (2022-10-07-DSTA-Technical_Specifications.odt) pour plus d'informations.

In [3]:
# Importation of Python libraries
import pandas as pd
from IPython.display import display
import requests
import urllib.request
import json
import ssl
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import column, row
from bokeh.client import push_session
from bokeh.models import HoverTool, WMTSTileSource, LinearColorMapper, LabelSet, Label
from bokeh.tile_providers import STAMEN_TERRAIN_RETINA, get_provider
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.server.server import Server
from bokeh.io import curdoc
import numpy as np
import folium

In [4]:
# Header
ssl._create_default_https_context = ssl._create_unverified_context
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]

In [5]:
# Provide your API key in the query string!!!
params = {
  'api_key': '9d2955c1-020a-4343-a6d7-c2f04450735b'
}
end_point = [
    "flights?",
    "airlines?",
    "schedules?"
]

In [ ]:
# Real time flight tracking URL:
api_base = 'http://airlabs.co/api/v9/'

# Loading of the data into a JSON named 'flights.json':
api_result = requests.get(api_base+end_point[0], params)
api_response = api_result.json()
api_flights = api_response["response"]

with open("flights.json", "w") as fp:
    json.dump(api_flights, fp, indent=4, sort_keys=True)

# Load the JSON file into a dataframe:
df = pd.read_json("flights.json")
# Control that it is working by printing the dataframe:
#display(df)

# Convert to Pandas Dataframe
flights_df = pd.DataFrame(df, columns = ['lat', 'lng', 'alt', 'flight_icao', 'dep_iata', 'arr_iata', 'airline_iata'])
display(flights_df)


Génération d'une carte en se basant sur la méthodologie de Mercator.
_______________________________________________________________________________________

France Coordonnées géographiques
_______________________________________________________________________________________

La France est située à la latitude 46.227638 et à la longitude 2.213749. Ce pays fait partie de l'Europe et de l'hémisphère nord.

Coordonnées décimales (Norme simplifiée)
46.227638, 2.213749 (Lat, long)

Coordonnées DD (Degrés décimaux)
46.2276° N 2.2137° E

Coordonnées SGD (Degrés, minutes et secondes)
46°13'39.5'' N 2°12.825' E

Le système de coordonnées géographiques permet de localiser n'importe quel endroit dans le monde en utilisant sa latitude et sa longitude. La latitude est la position par rapport à l'équateur, spécifiant la position nord-sud. La longitude spécifie la position est-ouest mesurée à partir d'un méridien de référence (généralement le premier méridien de Greenwich). La latitude et la longitude de la France ont été calculées sur la base du datum géodésique WGS84 (World Geodesic System 1984).

In [ ]:
# Convert to Pandas Dataframe
flights_df = pd.DataFrame(df, columns = ['lat', 'lng', 'alt', 'flight_icao', 'dep_iata', 'arr_iata', 'airline_iata'])
display(flights_df)

# Convert France GPS coordinates to Web Mercator
x_range, y_range = ((-6.403614, 12.229198), (40.587984, 53.007425))

# Convert All GPS coordinates to Web Mercator
def mercator(df, lat="lat", lon="lng"):
    radius = 6378137 # radius of the earth in meters
    df["x"] = df[lon] * (radius * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * radius
    return df

# Flights tracking main function
def flights_tracker(geo):
    # Bokeh column data source
    mercator_df = mercator(flights_df)
    flights_source = ColumnDataSource(mercator_df)
    display(flights_source)

    def streaming():
        # Map coordinates:
        # Objet MAP: https://docs.bokeh.org/en/latest/docs/reference/tile_providers.html
        map = figure(tools = 'pan, zoom_in, zoom_out, wheel_zoom, reset, save',
        x_range = x_range, y_range = y_range, x_axis_type = 'mercator',
          y_axis_type = 'mercator', sizing_mode = 'scale_width', plot_height = 300)

        # Add planes geolocalisation and informative labels object:
        infos = HoverTool()
        infos.tooltips = [('Flight', '@flight_icao'), ('Altitude', '@alt'),
         ('Departure', '@dep_iata'), ('Arrival', '@arr_iata'), ('Airline', '@airline_iata')]
        labels = LabelSet(x = 'x', y = 'y', text = 'flight_icao', level = 'glyph', x_offset = 5,
         y_offset = 5, source = flights_source, render_mode = 'canvas',
         background_fill_color = 'skyblue', text_color = 'black', text_font_size = '8pt')

        # Add the WMTS tile source to the map object:
        map_provider = (get_provider(STAMEN_TERRAIN_RETINA))
        map.add_tile(map_provider) # bokeh.tile_providers.StamenTerrainRetina
        map.add_tools(infos)                # Fucking doesn't work!!!
        map.circle('x', 'y', source = flights_source, fill_color = "blue", size = 10, fill_alpha = 0.8, line_width = 0.5)
        map.add_layout(labels)

        curdoc().title = "Flights Tracker"
        curdoc().add_root(map)
        session = push_session(curdoc())
        session.show()

#Display the Map in the browser:
#show(map)

# Server initialization for Bokeh Application (Flights Tracker) on port 48010:
# To run through bash terminal: bokeh serve --show FlightsTracker.py
apps = {'/': Application(FunctionHandler(flights_tracker))}
server = Server(apps, port=48010) # Any unassigned port will do
server.io_loop.start()